In [24]:
import pandas as pd
import json

# Prepare the result data

# Helper function to parse the answer
def parse_answer(ans):
    return json.loads(ans.strip("```").lstrip("json").replace("\n", "").replace(" ", "")).get("가장_적합한_HS_코드", "").rjust(6, "0")

# Load CSV
result = pd.read_csv('../results/coupang_result_v3_kor_20241208_182113.csv')

# Parse the answer
result['Result'] = result['Answer'].apply(parse_answer)

result = result.drop(columns = ['CleanDescription', 'Best3', 'Answer'])

result[:3]

,ProductName,Result
0,"품목명: 오뚜기 흰밥, 210g, 24개",100640
1,"품목명: 신라면 120g, 30개",190230
2,"품목명: 스팸 25% 라이트, 9개",150420


In [43]:
import ast
file_path = 'whole_ground_truth.txt'

with open(file_path, 'r') as file:
    file_content = file.read()

ground_truth = ast.literal_eval(file_content)
ground_truth[10]

{'가장_적합한_HS_코드': '190230',
 '가장_적합한_HS_항목명': 'Pasta,cookedorotherwiseprepared',
 '판단_근거': "비비고왕교자는기본적으로물과밀가루로만든교자피와속재료(고기,야채등)로이루어져있으며,냉동상태로제공되는제품이다.이러한특성은HS코드1902.30인'Pasta,cookedorotherwiseprepared'에가장잘맞는다.이코드는요리되거나다른방식으로준비된파스타를포함하고있으며,이는냉동교자가요리전또는요리후냉동처리되는것을포함한다.따라서이제품에는이HS코드가가장적합하다.",
 '상품_이름': '품목명: 비비고 왕교자 (냉동), 1.05kg, 2개',
 'index': 10}

In [16]:
gt_df = pd.DataFrame(ground_truth).drop(columns = ['가장_적합한_HS_항목명', '판단_근거'])
gt_df[:3]

,가장_적합한_HS_코드,상품_이름,index
0,190410,"품목명: 오뚜기 흰밥, 210g, 24개",0
1,190230,"품목명: 신라면 120g, 30개",1
2,160241,"품목명: 스팸 25% 라이트, 9개",2


In [21]:
gt_df = gt_df.sort_values(by='index').drop(columns = ['index'])

In [29]:
# Prepare the ground truth data

gt_df['가장_적합한_HS_코드'] = gt_df['가장_적합한_HS_코드'].apply(lambda x: str(x).rjust(6, "0"))
gt_df = gt_df.drop(columns = ['상품_이름'])

In [ ]:
result = result[:860]
print(type(result))
print(type(gt_df))

<class 'pandas.core.frame.DataFrame'>


In [32]:
merged_df = pd.concat([result, gt_df], axis=1)
merged_df

,ProductName,Result,가장_적합한_HS_코드
0,"품목명: 오뚜기 흰밥, 210g, 24개",100640,190410
1,"품목명: 신라면 120g, 30개",190230,190230
2,"품목명: 스팸 25% 라이트, 9개",150420,160241
3,"품목명: 동원 라이트 스탠다드 참치, 85g, 8개",160419,160414
4,"품목명: 동원 고추 참치, 85g, 8개",160419,160414
...,...,...,...
855,"품목명: 밥스레드밀 오가닉 통밀가루, 2.27kg, 1개",110630,110290
856,"품목명: 데체코 올리오 엑스트라 버진 디 올리바, 1L, 1개",150920,001509
857,"품목명: 오뚜기 핫케이크 믹스, 1kg, 2개",190590,190120
858,"품목명: 바삭하고 고소한 튀김가루 1KG, 1개",110630,190120


In [33]:
for i, row in merged_df.iterrows():
    # Create a dictionary from the row
    result = merged_df.loc[i, "Result"]
    gt = merged_df.loc[i, "가장_적합한_HS_코드"]
    merged_df.loc[i, "Match_2"] = result[:2] == gt[:2]
    merged_df.loc[i, "Match_4"] = result[:4] == gt[:4]
    merged_df.loc[i, "Match_6"] = result == gt

In [35]:
merged_df.to_csv('coupang_whole_comparison.csv', index=False)

In [36]:
# 각 매치별 True 비율 계산
for match_type in ["Match_2", "Match_4", "Match_6"]:
    true_ratio = merged_df[match_type].mean()
    print(f"{match_type} True 비율: {true_ratio:.2f}")

# 전체 True 비율 계산
total_true_ratio = merged_df[["Match_2", "Match_4", "Match_6"]].values.sum() / (3 * len(merged_df))
print(f"전체 True 비율: {total_true_ratio:.2f}")

Match_2 True 비율: 0.47
Match_4 True 비율: 0.37
Match_6 True 비율: 0.21
전체 True 비율: 0.35


In [4]:
import pandas as pd
sample = pd.read_csv('../../guidance_for_environmental_impact_factor_mapping_on_aws/assets/input/coupang_product_names_groceries_v3_kor.csv')
sample = sample.sample(20)

In [6]:
sample.to_csv('sample.csv', index=False)